In [1]:
!pip install librosa

     -------------------------------------- 253.7/253.7 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 919.9 kB/s eta 0:00:00
     ---------------------------------------- 63.0/63.0 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 659.3 kB/s eta 0:00:00
     ------------------------------------ 184.6/184.6 kB 657.9 kB/s eta 0:00:00
     ---------------------------------------- 75.1/75.1 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 28.1/28.1 MB 839.8 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
!pip install SpeechRecognition

     -------------------------------------- 32.8/32.8 MB 576.9 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import numpy as np
import pyaudio
import time
from scipy.io.wavfile import write
import librosa
import speech_recognition
import soundfile as sf

In [114]:
import pyaudio
import speech_recognition
from time import sleep


class Recorder():

    sampling_rate = 44100
    num_channels = 2
    sample_width = 4 # The width of each sample in bytes. Each group of ``sample_width`` bytes represents a single audio sample. 

    def pyaudio_stream_callback(self, in_data, frame_count, time_info, status):
        self.raw_audio_bytes_array.extend(in_data)
        return (in_data, pyaudio.paContinue)

    def start_recording(self):

        self.raw_audio_bytes_array = bytearray()

        pa = pyaudio.PyAudio()
        self.pyaudio_stream = pa.open(format=pyaudio.paInt16,
                                      channels=self.num_channels,
                                      rate=self.sampling_rate,
                                      input=True,
                                      stream_callback=self.pyaudio_stream_callback)

        self.pyaudio_stream.start_stream()

    def stop_recording(self):

        self.pyaudio_stream.stop_stream()
        self.pyaudio_stream.close()

        speech_recognition_audio_data = speech_recognition.AudioData(self.raw_audio_bytes_array,
                                                                     self.sampling_rate,
                                                                     self.sample_width)
        return speech_recognition_audio_data


if __name__ == '__main__':

    recorder = Recorder()

    # start recording
    recorder.start_recording()

    # say something interesting...
    sleep(3)

    # stop recording
    speech_recognition_audio_data = recorder.stop_recording()

    # convert the audio represented by the ``AudioData`` object to
    # a byte string representing the contents of a WAV file
    wav_data = speech_recognition_audio_data.get_wav_data()

In [115]:
# Writing to file
with open("sound.wav", "wb") as file1:
    # Writing data to a file
    file1.write(wav_data)

In [116]:
audio_file = librosa.load('sound.wav')
y, sr = audio_file


In [117]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

In [118]:
tempo

117.45383522727273

In [45]:
wav_array = np.frombuffer(wav_data, dtype=np.float32)

In [48]:
no_nan = np.nan_to_num(wav_array)

In [49]:
np.isnan(np.min(no_nan))

False

In [50]:
tempo, beat_frames = librosa.beat.beat_track(y=no_nan, sr=44100)

C:\Users\Cmgoe\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:362: RuntimeWarning: overflow encountered in cast
  stft_matrix[..., :off_start] = fft.rfft(fft_window * y_frames_pre, axis=-2)
C:\Users\Cmgoe\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:378: RuntimeWarning: overflow encountered in cast
  stft_matrix[..., bl_s + off_start : bl_t + off_start] = fft.rfft(
C:\Users\Cmgoe\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:2821: RuntimeWarning: overflow encountered in square
  np.abs(


ParameterError: Input must be finite

In [40]:
librosa.output.write_wav('beeb.wav', wav_data, 44100)

AttributeError: No librosa attribute output

In [2]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=1,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


In [34]:
speech_recognition_audio_data = speech_recognition.AudioData(frames,
                                                                     RATE,
                                                                     4)

In [35]:
speech_recognition_audio_data.get_wav_data()

TypeError: memoryview: a bytes-like object is required, not 'list'

In [21]:
np_arr = np.array([])
for chunk in frames:
    np_arr = np.append(np_arr, np.frombuffer(chunk, dtype=np.float32))

In [22]:
type(np_arr)

numpy.ndarray

In [23]:
tempo, beat_frames = librosa.beat.beat_track(y=np_arr, sr=RATE)

ParameterError: Audio buffer is not finite everywhere

In [4]:
play=pyaudio.PyAudio()
stream_play=play.open(format=FORMAT,
                      channels=CHANNELS,
                      rate=RATE,
                      output=True)
for data in frames: 
    stream_play.write(data)
stream_play.stop_stream()
stream_play.close()
play.terminate()

In [23]:

class AudioHandler(object):
    def __init__(self):
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024 * 1024
        self.p = None
        self.stream = None

    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  input_device_index=1,
                                  stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK)

    def stop(self):
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        print(numpy_array)
        tempo, beat_frames = librosa.beat.beat_track(y=numpy_array, sr=self.RATE)
        print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            time.sleep(4.0)


audio = AudioHandler()
audio.start()     # open the the stream
audio.mainloop()  # main operations with librosa
audio.stop()

[0.00036621 0.00021362 0.00021362 ... 0.00247192 0.00231934 0.00204468]
Estimated tempo: 120.19 beats per minute
[0.0020752  0.00253296 0.00253296 ... 0.00164795 0.00180054 0.0015564 ]
Estimated tempo: 139.67 beats per minute
[ 0.00076294 -0.00061035 -0.0015564  ...  0.00131226 -0.00012207
 -0.00088501]
Estimated tempo: 120.19 beats per minute
[-0.00100708 -0.00109863 -0.00097656 ... -0.00482178 -0.00521851
 -0.00592041]
Estimated tempo: 120.19 beats per minute
[-0.00613403 -0.00662231 -0.00637817 ...  0.0027771   0.00305176
  0.00299072]
Estimated tempo: 0.00 beats per minute
[2.9907227e-03 3.0517578e-03 3.1127930e-03 ... 4.8828125e-04 9.1552734e-05
 6.1035156e-05]
Estimated tempo: 120.19 beats per minute
[ 0.00021362 -0.00030518  0.         ... -0.0045166  -0.00479126
 -0.00442505]
Estimated tempo: 120.19 beats per minute
[-0.00454712 -0.00479126 -0.00460815 ...  0.00119019  0.00137329
  0.00134277]
Estimated tempo: 0.00 beats per minute
[0.00134277 0.00134277 0.00134277 ... 0.000885

[-3.0517578e-05 -6.1035156e-05  1.2207031e-04 ...  2.0141602e-03
  1.9531250e-03  2.5329590e-03]
Estimated tempo: 139.67 beats per minute
[0.00244141 0.00234985 0.00250244 ... 0.00231934 0.00234985 0.0022583 ]
Estimated tempo: 120.19 beats per minute
[0.00192261 0.00201416 0.00180054 ... 0.00265503 0.00274658 0.00302124]
Estimated tempo: 120.19 beats per minute
[ 0.00286865  0.00274658  0.00311279 ... -0.00109863 -0.00109863
 -0.00100708]
Estimated tempo: 0.00 beats per minute
[-0.00079346 -0.00100708 -0.00109863 ...  0.00344849  0.0032959
  0.00317383]
Estimated tempo: 136.00 beats per minute
[0.00338745 0.00326538 0.00366211 ... 0.00369263 0.00366211 0.00302124]
Estimated tempo: 0.00 beats per minute
[0.00299072 0.00296021 0.00335693 ... 0.00424194 0.0043335  0.00421143]
Estimated tempo: 0.00 beats per minute
[ 0.00405884  0.00405884  0.00424194 ... -0.00091553 -0.00115967
 -0.00088501]
Estimated tempo: 0.00 beats per minute
[-0.00079346 -0.00085449 -0.0010376  ... -0.00418091 -0.004

[ 0.00045776  0.00018311  0.00024414 ... -0.00238037 -0.00219727
 -0.00241089]
Estimated tempo: 120.19 beats per minute
[-0.00268555 -0.00247192 -0.00253296 ...  0.00097656  0.00097656
  0.00106812]
Estimated tempo: 0.00 beats per minute
[ 0.00112915  0.00112915  0.00097656 ... -0.00091553 -0.00079346
 -0.00091553]
Estimated tempo: 139.67 beats per minute
[-0.00167847 -0.00106812 -0.00119019 ... -0.00082397 -0.00079346
 -0.00082397]
Estimated tempo: 120.19 beats per minute
[-0.00085449 -0.00094604 -0.00073242 ...  0.00231934  0.00247192
  0.00247192]
Estimated tempo: 120.19 beats per minute
[ 0.00222778  0.00234985  0.00222778 ... -0.00158691 -0.00140381
 -0.0012207 ]
Estimated tempo: 0.00 beats per minute
[-0.00119019 -0.00131226 -0.00115967 ...  0.00296021  0.00308228
  0.00320435]
Estimated tempo: 120.19 beats per minute
[0.00292969 0.00299072 0.00326538 ... 0.00015259 0.00024414 0.00018311]
Estimated tempo: 0.00 beats per minute
[ 3.0517578e-05  1.5258789e-04  2.7465820e-04 ... -4.

[-0.00234985 -0.00259399 -0.00250244 ...  0.00201416  0.00231934
  0.0022583 ]
Estimated tempo: 0.00 beats per minute
[ 0.00189209  0.00216675  0.00238037 ... -0.00106812 -0.00067139
 -0.00048828]
Estimated tempo: 120.19 beats per minute
[-0.00057983 -0.00067139 -0.00082397 ...  0.00057983  0.0010376
  0.00073242]
Estimated tempo: 120.19 beats per minute
[0.0007019  0.00057983 0.00027466 ... 0.0007019  0.00079346 0.00085449]
Estimated tempo: 136.00 beats per minute
[ 0.00100708  0.00082397  0.00082397 ... -0.00100708 -0.00152588
 -0.00146484]
Estimated tempo: 120.19 beats per minute
[-0.0012207  -0.00112915 -0.00119019 ...  0.0015564   0.00119019
  0.00128174]
Estimated tempo: 120.19 beats per minute
[1.2512207e-03 1.4038086e-03 1.4953613e-03 ... 0.0000000e+00 9.1552734e-05
 1.2207031e-04]
Estimated tempo: 120.19 beats per minute
[ 0.0000000e+00  1.5258789e-04  2.7465820e-04 ... -4.2724609e-04
 -3.0517578e-04 -3.0517578e-05]
Estimated tempo: 120.19 beats per minute
[-3.9672852e-04 -5.4

Estimated tempo: 120.19 beats per minute
[ 0.00015259 -0.00079346 -0.00100708 ... -0.0020752  -0.00152588
 -0.00094604]
Estimated tempo: 132.51 beats per minute
[-0.00115967 -0.00143433 -0.00170898 ...  0.00088501  0.00222778
  0.00131226]
Estimated tempo: 139.67 beats per minute
[ 0.00088501  0.00048828  0.00100708 ... -0.00067139 -0.00097656
 -0.00067139]
Estimated tempo: 132.51 beats per minute
[-0.00027466  0.00061035  0.00054932 ... -0.00241089 -0.00265503
 -0.00219727]
Estimated tempo: 136.00 beats per minute
[-0.00210571 -0.00192261 -0.00167847 ...  0.00265503  0.00317383
  0.00228882]
Estimated tempo: 120.19 beats per minute
[0.00222778 0.00164795 0.00100708 ... 0.0012207  0.0012207  0.00167847]
Estimated tempo: 136.00 beats per minute
[0.00222778 0.00149536 0.00106812 ... 0.00192261 0.00134277 0.00091553]
Estimated tempo: 120.19 beats per minute
[ 0.00036621 -0.00024414 -0.00082397 ...  0.00186157  0.00180054
  0.0020752 ]
Estimated tempo: 120.19 beats per minute
[ 0.00253296 

...

Estimated tempo: 139.67 beats per minute
[ 0.00628662  0.00509644  0.00405884 ...  0.0020752  -0.00891113
 -0.00942993]
Estimated tempo: 139.67 beats per minute
[-0.00332642 -0.0012207  -0.01504517 ... -0.00408936  0.00021362
  0.00238037]
Estimated tempo: 120.19 beats per minute
[ 0.00784302  0.01443481  0.0144043  ... -0.01010132 -0.00961304
 -0.01223755]
Estimated tempo: 120.19 beats per minute
[-0.01599121 -0.01855469 -0.01589966 ... -0.00161743 -0.00537109
 -0.00717163]
Estimated tempo: 139.67 beats per minute
[-0.00564575 -0.0039978  -0.00405884 ... -0.00186157 -0.00268555
 -0.00314331]
Estimated tempo: 120.19 beats per minute
[-0.00527954 -0.00888062 -0.00991821 ...  0.00402832 -0.00039673
 -0.00668335]
Estimated tempo: 139.67 beats per minute
[-0.01397705 -0.01742554 -0.01437378 ... -0.00259399 -0.00161743
  0.00128174]
Estimated tempo: 120.19 beats per minute
[ 0.00518799  0.0067749   0.00695801 ... -0.00952148 -0.01052856
 -0.01037598]
Estimated tempo: 120.19 beats per minute

[ 0.0133667   0.00778198  0.00393677 ...  0.00375366  0.00186157
 -0.00073242]
Estimated tempo: 120.19 beats per minute
[-0.00036621  0.00469971  0.0116272  ... -0.0017395   0.00216675
  0.00546265]
Estimated tempo: 120.19 beats per minute
[ 0.00546265  0.00167847 -0.00152588 ...  0.00143433  0.00036621
 -0.00097656]
Estimated tempo: 120.19 beats per minute
[-2.1362305e-03 -1.7395020e-03 -1.0375977e-03 ...  1.7395020e-03
  4.5776367e-04 -3.0517578e-05]
Estimated tempo: 139.67 beats per minute
[ 0.00067139  0.00219727  0.00305176 ... -0.00088501  0.0145874
  0.0199585 ]
Estimated tempo: 139.67 beats per minute
[ 1.1291504e-02 -6.1035156e-05 -2.5939941e-03 ... -7.6904297e-03
 -7.0495605e-03 -6.8359375e-03]
Estimated tempo: 120.19 beats per minute
[-0.00610352 -0.00393677 -0.00131226 ...  0.0017395   0.00631714
  0.00775146]
Estimated tempo: 120.19 beats per minute
[ 0.00512695  0.00250244  0.00219727 ...  0.0005188  -0.00387573
 -0.00534058]
Estimated tempo: 136.00 beats per minute
[-0.0

[ 0.00079346  0.00015259 -0.00021362 ...  0.          0.00033569
  0.00027466]
Estimated tempo: 120.19 beats per minute
[ 0.00045776 -0.00030518 -0.00094604 ...  0.00158691  0.00097656
  0.00057983]


KeyboardInterrupt: 

Estimated tempo: 120.19 beats per minute


In [ ]:
p = pyaudio.PyAudio()

In [14]:
p.get_device_info_by_index(1)

{'index': 1,
 'structVersion': 2,
 'name': 'Microphone (Realtek USB Audio)',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}